In [1]:
import ee

# Initialize the Earth Engine API
ee.Initialize()


In [18]:
import geopandas as gpd
import json

# Load the shapefile using GeoPandas to define the area of interest (AOI)
gdf = gpd.read_file(r"C:\Users\konst\Documents\Hiwi\mw3\drought_indicies\data\Untersuchungsgebiete\002a_Sen1-Subset_Eifel.shp")

# Convert the shapefile to GeoJSON format for compatibility with Earth Engine
geojson = gdf.to_json()

# Convert the GeoJSON geometry to an Earth Engine Geometry object
aoi = ee.Geometry(json.loads(geojson)['features'][0]['geometry'])

# Define the date range for filtering the datasets
start_date = '2013-01-01'
end_date = '2025-01-31'

# Define a cloud masking function for filtering out clouds and cloud shadows in images
def mask_clouds(image):
    # Select the Fmask band, which contains cloud-related information
    fmask = image.select('Fmask')
    
    # Extract the cloud bit (bit 1) and shadow bit (bit 3) from the Fmask band
    cloud_bit = fmask.rightShift(1).bitwiseAnd(1)  # Bit 1 (cloud)
    adjacent_bit = fmask.rightShift(2).bitwiseAnd(1) # Bit 2 (adjacent to clouds, unused)
    shadow_bit = fmask.rightShift(3).bitwiseAnd(1)  # Bit 3 (shadow)
    
    # Create a mask that excludes pixels where cloud_bit or shadow_bit is 1 (cloudy or shadowed)
    mask = cloud_bit.eq(0).And(shadow_bit.eq(0))
    
    # Return the image with the mask applied
    return image.updateMask(mask)

# Load and filter the Landsat-based HLS (HLSL30) dataset
hlsl30 = (
    ee.ImageCollection('NASA/HLS/HLSL30/v002')  # Landsat-based HLS dataset
    .filterBounds(aoi)                         # Filter images by the AOI
    .filterDate(start_date, end_date)          # Filter images by the date range
    .map(mask_clouds)                          # Apply the cloud masking function
)

# Load and filter the Sentinel-2-based HLS (HLSS30) dataset
hlss30 = (
    ee.ImageCollection('NASA/HLS/HLSS30/v002')  # Sentinel-2-based HLS dataset
    .filterBounds(aoi)                          # Filter images by the AOI
    .filterDate(start_date, end_date)           # Filter images by the date range
    .map(mask_clouds)                           # Apply the cloud masking function
)

hlss30_renamed = hlss30.map(lambda img: img.select(
    ['B2', 'B3', 'B4', 'B8A', 'B11', 'B12', 'B10','Fmask'],
    ['Blue','Green','Red','NIR','SWIR1','SWIR2','Cirrus','Fmask']
))

hlsl30_renamed = hlsl30.map(lambda img: img.select(
    ['B2','B3','B4','B5','B6','B7','B9','Fmask'],
    ['Blue','Green','Red','NIR','SWIR1','SWIR2','Cirrus','Fmask']
))

hls_col = hlss30_renamed.merge(hlsl30_renamed)

# Debugging: Print the number of images in the merged collection and an example image
print('Number of images in the merged HLS collection:', hls_col.size().getInfo())
print('Example image:', hls_col.first().getInfo())

# Function to group the collection by month and calculate the median for each month
def calculate_monthly_medians(collection, start_date, end_date):
    # Generate a list of months within the date range
    months = ee.List.sequence(0, ee.Date(end_date).difference(ee.Date(start_date), 'month').round())

    # Function to calculate the median for a specific month
    def get_monthly_median(month):
        # Define the start and end dates for the current month
        start_month = ee.Date(start_date).advance(month, 'month')
        end_month = start_month.advance(1, 'month')
        
        # Filter the collection to include only images within the current month
        filtered = collection.filterDate(start_month, end_month)
        
        # Calculate the median for the filtered collection and set metadata
        median = filtered.median().set({
            'month': start_month.format('YYYY-MM'),  # Add month metadata
            'system:time_start': start_month.millis()  # Add timestamp metadata
        })
        
        return median

    # Map the median calculation function over the list of months
    return ee.ImageCollection.fromImages(months.map(get_monthly_median))

# Calculate the monthly median images for the merged collection
monthly_medians = calculate_monthly_medians(hls_col, start_date, end_date)

# Debugging: Print the number of monthly median images
print('Number of monthly median images:', monthly_medians.size().getInfo())

# Function to export the monthly median images to Google Drive
def export_monthly_medians(collection, aoi, folder_name='HLS_Monthly_Medians', scale=30):
    # Convert the collection to a list of images for iteration
    collection_list = collection.toList(collection.size())
    
    # Iterate through the collection and export each image
    for i in range(collection.size().getInfo()):
        # Get the current image
        image = ee.Image(collection_list.get(i))
        # Retrieve the month metadata
        month = image.get('month').getInfo()
        
        # Create and start the export task
        task = ee.batch.Export.image.toDrive(
            image=image.clip(aoi),  # Clip the image to the AOI
            description=f'HLS_Monthly_Median_{month}',  # Task description
            folder=folder_name,  # Google Drive folder name
            fileNamePrefix=f'HLS_Monthly_Median_{month}',  # File name prefix
            region=aoi.bounds().getInfo()['coordinates'],  # AOI boundaries
            scale=scale,  # Spatial resolution
            maxPixels=1e13  # Maximum allowable pixels
        )
        task.start()
        print(f'Exporting HLS Monthly Median for {month} to Google Drive...')

# Export the monthly median images
export_monthly_medians(monthly_medians, aoi)


Number of images in the merged HLS collection: 3428
Example image: {'type': 'Image', 'bands': [{'id': 'Blue', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -3.2768, 'max': 3.2767}, 'dimensions': [3660, 3660], 'crs': 'EPSG:32631', 'crs_transform': [30, 0, 699960, 0, -30, 5600040]}, {'id': 'Green', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -3.2768, 'max': 3.2767}, 'dimensions': [3660, 3660], 'crs': 'EPSG:32631', 'crs_transform': [30, 0, 699960, 0, -30, 5600040]}, {'id': 'Red', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -3.2768, 'max': 3.2767}, 'dimensions': [3660, 3660], 'crs': 'EPSG:32631', 'crs_transform': [30, 0, 699960, 0, -30, 5600040]}, {'id': 'NIR', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -3.2768, 'max': 3.2767}, 'dimensions': [3660, 3660], 'crs': 'EPSG:32631', 'crs_transform': [30, 0, 699960, 0, -30, 5600040]}, {'id': 'SWIR1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -

In [1]:
import rioxarray as rxr
import xarray as xr
import glob
import os
import re
import pandas as pd
from datetime import datetime
import numpy as np

# 1) Folder with all your .tif files
folder = "C:/Users/konst/Documents/Hiwi/mw3/master_thesis/data/satellite_data/hls_eifel/raw/"

# 2) Get list of all .tif files
tif_files = sorted(glob.glob(os.path.join(folder, "*.tif")))

# 3) Define band names mapping
band_names = {
    1: 'Blue',
    2: 'Green',
    3: 'Red',
    4: 'NIR',
    5: 'SWIR1',
    6: 'SWIR2',
    7: 'Cirrus',
    8: 'Fmask'
}

# 4) Function to extract date from filename
def extract_date(filename):
    # Adjust this regex pattern to match your filename format
    pattern = r'(\d{4}-\d{2})'
    match = re.search(pattern, filename)
    if match:
        return pd.to_datetime(match.group(1))
    raise ValueError(f"Could not extract date from filename: {filename}")

# 5) Read all files and create a list of datasets
datasets = []
for file in tif_files:
    # Read the file
    ds = rxr.open_rasterio(file)
    
    # Extract date from filename
    date = extract_date(file)
    
    # Convert band dimension to variables
    ds = ds.to_dataset(dim='band')
    
    # Rename variables using the band_names mapping
    ds = ds.rename({i: band_names[i] for i in range(1, 9)})
    
    # Add time coordinate
    ds = ds.expand_dims(time=[date])
    
    datasets.append(ds)

# 6) Combine all datasets
combined_ds = xr.concat(datasets, dim='time')

# 7) Add metadata
combined_ds.attrs['title'] = 'Combined satellite data'
combined_ds.attrs['creation_date'] = datetime.now().strftime('%Y-%m-%d')

# Loop through each variable in the dataset
for var in combined_ds.data_vars:
    combined_ds[var] = combined_ds[var].where((combined_ds[var] > 0) & np.isfinite(combined_ds[var]), np.nan)

# 7) Save to NetCDF
output_file = os.path.join(folder, 'HLS_Monthly_Medians_2013-04_2025-01.nc')
combined_ds.to_netcdf(output_file)

# 8) Close all datasets
for ds in datasets:
    ds.close()


In [7]:
combined_ds.keys()

KeysView(<xarray.Dataset> Size: 8GB
Dimensions:      (time: 142, y: 612, x: 1394)
Coordinates:
  * time         (time) datetime64[ns] 1kB 2013-04-01 2013-05-01 ... 2025-01-01
  * x            (x) float64 11kB 6.284 6.284 6.285 6.285 ... 6.659 6.659 6.66
  * y            (y) float64 5kB 50.64 50.64 50.64 50.64 ... 50.48 50.48 50.48
    spatial_ref  int64 8B 0
Data variables:
    1            (time, y, x) float64 969MB nan nan nan nan ... nan nan nan nan
    2            (time, y, x) float64 969MB nan nan nan nan ... nan nan nan nan
    3            (time, y, x) float64 969MB nan nan nan nan ... nan nan nan nan
    4            (time, y, x) float64 969MB nan nan nan nan ... nan nan nan nan
    5            (time, y, x) float64 969MB nan nan nan nan ... nan nan nan nan
    6            (time, y, x) float64 969MB nan nan nan nan ... nan nan nan nan
    7            (time, y, x) float64 969MB nan nan nan nan ... nan nan nan nan
    8            (time, y, x) float64 969MB nan nan nan nan ...